<a href="https://colab.research.google.com/github/appliedcode/mthree-c422/blob/mthree-c422-Likhitha/02_Vocabulary_and_Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
from spacy.matcher import Matcher, PhraseMatcher

# Load English tokenizer, POS tagger, parser, NER
nlp = spacy.load("en_core_web_sm")


In [3]:
matcher = Matcher(nlp.vocab)



In [5]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

# Define patterns
pattern1 = [{'LOWER': 'solarpower'}]
pattern2 = [{'LOWER': 'solar'}, {'LOWER': 'power'}]
pattern3 = [{'LOWER': 'solar'}, {'IS_PUNCT': True}, {'LOWER': 'power'}]

# Add patterns to matcher (updated for spaCy v3+)
matcher.add('SolarPower', [pattern1, pattern2, pattern3])

doc = nlp("The Solar Power industry continues to grow. solarpower, solar power, and solar-power are common terms.")

matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print(match_id, span.text)


8656102463236116519 Solar Power
8656102463236116519 solarpower
8656102463236116519 solar power
8656102463236116519 solar-power


In [6]:
text = "The Solar Power industry continues to grow as demand for solarpower increases. Solar-power cars are gaining popularity."
doc = nlp(text)

found_matches = matcher(doc)
for match_id, start, end in found_matches:
    span = doc[start:end]
    print(match_id, nlp.vocab.strings[match_id], start, end, span.text)


8656102463236116519 SolarPower 1 3 Solar Power
8656102463236116519 SolarPower 10 11 solarpower
8656102463236116519 SolarPower 13 16 Solar-power


In [9]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
doc = nlp("Solarpower, solar power, and solar-power are all valid terms.")

matcher = Matcher(nlp.vocab)

# Safe remove
try:
    matcher.remove('SolarPower')
except ValueError:
    pass

# Define patterns
pattern1 = [{'LOWER': 'solarpower'}]
pattern2 = [{'LOWER': 'solar'}, {'IS_PUNCT': True, 'OP': '*'}, {'LOWER': 'power'}]

# Add patterns correctly for spaCy v3+
matcher.add('SolarPower', [pattern1, pattern2])

# Find matches
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print(match_id, nlp.vocab.strings[match_id], start, end, span.text)


8656102463236116519 SolarPower 0 1 Solarpower
8656102463236116519 SolarPower 2 4 solar power
8656102463236116519 SolarPower 6 9 solar-power


In [11]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

# Remove existing pattern safely
try:
    matcher.remove('SolarPower')
except ValueError:
    pass

# Define all variations
pattern1 = [{'LOWER': 'solarpower'}]
pattern2 = [{'LOWER': 'solar'}, {'IS_PUNCT': True, 'OP': '*'}, {'LOWER': 'power'}]
pattern3 = [{'LOWER': 'solarpowered'}]
pattern4 = [{'LOWER': 'solar'}, {'IS_PUNCT': True, 'OP': '*'}, {'LOWER': 'powered'}]

# Correct syntax
matcher.add('SolarPower', [pattern1, pattern2, pattern3, pattern4])

# Test doc
doc2 = nlp("Solar-powered energy runs solarpower and solar powered cars, as well as solarpowered machines.")

# Find and print matches
found_matches = matcher(doc2)
for match_id, start, end in found_matches:
    span = doc2[start:end]
    print(match_id, nlp.vocab.strings[match_id], start, end, span.text)


8656102463236116519 SolarPower 0 3 Solar-powered
8656102463236116519 SolarPower 5 6 solarpower
8656102463236116519 SolarPower 7 9 solar powered
8656102463236116519 SolarPower 14 15 solarpowered


In [13]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")

pattern = [{'ORTH': '#'}, {}]  # Match a '#' followed by any token
matcher = Matcher(nlp.vocab)
matcher.add('HASHTAG', [pattern])  # ✅ correct syntax

doc = nlp("Use #AI to enhance productivity.")
matches = matcher(doc)

for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)


#AI


In [14]:
# Reset matcher
matcher = PhraseMatcher(nlp.vocab)

# Example terminology list
phrase_list = ['voodoo economics', 'supply-side economics', 'trickle-down economics', 'free-market economics']

# Convert phrases to Doc objects
phrase_patterns = [nlp(text) for text in phrase_list]

# Add to PhraseMatcher
matcher.add('VoodooEconomics', None, *phrase_patterns)


In [16]:
text = """
Reaganomics refers to the economic policies promoted by U.S. President Ronald Reagan.
These policies are commonly associated with supply-side economics, trickle-down theory, and tax cuts.
"""
doc3 = nlp(text)

matches = matcher(doc3)
for match_id, start, end in matches:
    span = doc3[start:end]
    print(span.text)


supply-side economics


In [17]:
for match_id, start, end in matches:
    span = doc3[start:end]
    print(match_id, nlp.vocab.strings[match_id], start, end, span.text)


3473369816841043438 VoodooEconomics 21 25 supply-side economics


In [18]:
# Example: Get context before and after a match
print(doc3[665:685])        # Before and after 5th match (starts at 673)
print(doc3[2975:2995])      # Before and after 6th match (starts at 2985)


In [20]:
if len(matches) >= 5:
    match_start = matches[4][1]
    for sent in sents:
        if match_start < sent.end:
            print("Sentence containing the 5th match:")
            print(sent.text)
            break
else:
    print("Less than 5 matches found.")


Less than 5 matches found.
